In [1]:
import ee
import geemap

In [ ]:
ee.Authenticate()

In [ ]:
ee.Initialize(project="ee-bau-tubitak-ndvi")

In [ ]:
parsel_data = ee.FeatureCollection("projects/ee-bau-tubitak-ndvi/assets/DestekParselleri")
image_data = ee.ImageCollection("COPERNICUS/S2")
year = 2018
month = 1

In [ ]:
def mask_clouds(image):
    QA60 = image.select(["QA60"])
    cloud_mask = QA60.bitwiseAnd(1 << 10).eq(0)
    return image.updateMask(cloud_mask)

def add_ndvi(input_image):
    nd = input_image.normalizedDifference(["B8", "B4"]).rename("ndvi")
    return input_image.addBands(nd)

In [ ]:
roi = parsel_data.geometry().convexHull()
roi_outline = ee.Image().paint(roi, 0, 1)
start_date = ee.Date.fromYMD(year, month, 1)
end_date = start_date.advance(1, "month")

rgb_viz_params = {
    "bands": ["B4", "B3", "B2"],
    "min": 0,
    "max": 3000
}

ndvi_viz_params = {
    "min": -1,
    "max": 1,
    "palette": ["red", "white", "green"],
}

In [ ]:
image = image_data \
    .filterDate(start_date, end_date) \
    .filterBounds(roi) \
    .map(add_ndvi) \
    .map(mask_clouds) \
    .median()

In [ ]:
Map = geemap.Map()

Map.add_layer(image, rgb_viz_params, "RGB Layer")
Map.add_layer(image.select("ndvi"), ndvi_viz_params, "Ndvi Layer")
Map.addLayer(roi_outline, {"palette": "Red"}, "Roi")
Map.centerObject(roi, 12)

Map

In [ ]:
def featureCollection_init(feature):
    tarimParse = feature.get("TarimParse")
    geometry = feature.geometry()
    return ee.Feature(geometry).set("TarimParse", tarimParse).set("mean_ndvi", [])

def collect_data(image, database):
    def map_function(feature):
        feature = ee.Feature(feature)
        mean_ndvi = image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=feature.geometry(),
            scale=30,
            maxPixels=1e9
        ).get("ndvi")
        
        mean_ndvi_array = ee.List(feature.get("mean_ndvi"))
        mean_ndvi_array = mean_ndvi_array.add(mean_ndvi)
        return feature.set("mean_ndvi", mean_ndvi_array)
    
    return database.map(map_function)

In [ ]:
database = parsel_data.distinct("TarimParse").map(featureCollection_init)
database = collect_data(image, database)


In [ ]:
print(database.limit(100).getInfo())